In [ ]:
import torch     # PyTorch
import datasets  # Huggingface Datasets; actual data

from IPython.display import display  # image display
import matplotlib.pyplot as plt      # image display
%matplotlib inline                   
import random
#Torchvision functions
from torchvision.transforms import ToTensor
import torchvision.transforms.functional as functional

In [ ]:
dataset = datasets.load_dataset("DrishtiSharma/Anime-Face-Dataset")

In [ ]:
#loading all images from dictionary to list
#all_images = dataset['train'][:63565]['image']
#resizing the first image
#resized_image = functional.resize(all_images[0], size=[64])
#saving the image to a torch tensor
#train_tensor = train_tensor = ToTensor()(resized_image).unsqueeze(0)
#for the rest of the images, resize then concatinate to that tensor, using dimension 0.
for i in range(27528, len(all_images)):
    resized_image = functional.resize(all_images[i], size=[64])
    train_tensor = torch.cat((train_tensor,ToTensor()(resized_image).unsqueeze(0)), 0)
    #if i%10000 == 0:  #I used these as a sanity check for my loop, ignore
    #    print('i = ', i)

#Saving the tensor so we dont run the above code multiple times.
torch.save(train_tensor, 'images.pt')
#If everything went well, we will see a tensor of size [63565, 3, 64, 64]:
train_tensor